In [18]:
import pickle
data = pickle.load(open(r'D:\PHD\Codes\AutoML\GraphNAP_V1.0\data\graph_anomaly\elliptic.dat', 'rb'))

In [19]:
print(data)

Data(edge_index=[2, 73248], num_nodes=46564, x=[46564, 93], y=[46564], train_mask=[46564], val_mask=[46564], test_mask=[46564])


In [20]:
import torch
train_mask =0
val_mask=0
test_mask=0
for i in range(len(data.y)):
    if data.train_mask[i]==torch.tensor(1):
        train_mask=train_mask+1
    if data.val_mask[i]==torch.tensor(1):
        val_mask=val_mask+1
    if data.test_mask[i]==torch.tensor(1):
        test_mask=test_mask+1
print("train mask count is:",train_mask)
print("val mask count is :",val_mask)
print("test mask count is :",test_mask)
print("total train+ val+test mask count is ",train_mask+val_mask+test_mask)

train mask count is: 21353
val mask count is : 8541
test mask count is : 16670
total train+ val+test mask count is  46564


In [ ]:
import torch
train =0
test=0
no_train=0
for i in range(len(data.y)):
    if data.train_mask[i]==torch.tensor(1):
        train=train+1
    else:
        no_train=no_train+1
print("train count",train)
print("train count",train)
print("train count",train+no_train)
train =0
test=0
no_train=0
for i in range(len(data.y)):
    if data.train_mask[i]==torch.tensor(1):
        train=train+1
    else:
        no_train=no_train+1
print("train count",train)
print("train count",train)
print("train count",train+no_train)

In [ ]:
from torch_geometric.datasets import Planetoid
dataset=Planetoid(root=r"D:\PHD\Codes\AutoML\GraphNAP_V1.0\data\Planetoid", name="Cora")

In [ ]:
print(dataset)